In [1]:
import sys, time
sys.path.append("..")

from carlasim.carla_client import CarlaClient
from carlasim.ego_car import EgoCar
from planner.stub_slam import StubSLAM
from planner.stub_global_planner import StubGlobalPlanner
from motion.path_position_finder import PathPositionFinder
import carla
from model.vehicle_pose import VehiclePose
from typing import List

import gi
gi.require_version('Gst', '1.0')
gi.require_version('GstRtspServer', '1.0')
from gi.repository import Gst, GLib
Gst.init(None)

/workspaces/crawler-poc/notebooks/../carlasim/video_streamer.py:1: PyGIWarning: Gst was imported without specifying a version first. Use gi.require_version('Gst', '1.0') before import to ensure that the right version gets loaded.
  from gi.repository import Gst


[]

In [2]:
StartPosition = ['Town07', -100, 0, 0, 0.0]
global_planner = StubGlobalPlanner()
global_planner.read_mission("./small_mission.dat")
client = CarlaClient(town=StartPosition[0])
ego = EgoCar(client)
ego.set_pose(StartPosition[1], StartPosition[2], StartPosition[3], StartPosition[4])
ego.set_power(0)
ego.set_steering(0)

In [ ]:
def redraw_path():
    world = client.get_world()

    for w in global_planner.get_all_poses():    
        world.debug.draw_string(carla.Location(w.x, w.y, 2), 'O', draw_shadow=False,
                                        color=carla.Color(r=255, g=0, b=0), life_time=120.0,
                                        persistent_lines=True)


Path Position Finder

In [ ]:

def mark_pose(path: List[VehiclePose], pos: int):
    redraw_path()
    world = client.get_world()
    world.debug.draw_string(carla.Location(path[pos].x, path[pos].y, 2), 'X', draw_shadow=False,
    color=carla.Color(r=0, g=255, b=0), life_time=120.0, persistent_lines=True)
    
    world.debug.draw_string(carla.Location(path[pos+1].x, path[pos+1].y, 2), 'X', draw_shadow=False,
    color=carla.Color(r=0, g=255, b=0), life_time=120.0, persistent_lines=True)

Testing cases

In [5]:
path = global_planner.get_all_poses()
slam = StubSLAM(ego)

position_finder = PathPositionFinder(path)
ego.set_pose(-100, 0, 0, 0)
time.sleep(0.5)
i = position_finder.find_next_pos(slam.estimate_ego_pose())
if i < 0:
    print ("error, should be > 0")
else:
    mark_pose(path, i)

in range for [0 - 1]
